In [2]:
import numpy as np
import pandas as pd
from xgboost import XGBRegressor
from sklearn.linear_model import LinearRegression
import matplotlib.pyplot as plt

In [3]:
timeseries=pd.read_csv('Store-Sales-Time-Series.csv')
timeseries['date'] = pd.to_datetime(timeseries['date'])
timeseries

,date,AUTOMOTIVE,BABY CARE,BEAUTY,BEVERAGES,BOOKS,BREAD/BAKERY,CELEBRATION,CLEANING,DAIRY,...,MAGAZINES,MEATS,PERSONAL CARE,PET SUPPLIES,PLAYERS AND ELECTRONICS,POULTRY,PREPARED FOODS,PRODUCE,SCHOOL AND OFFICE SUPPLIES,SEAFOOD
0,2013-01-01,0.0,0.0,2.0,810.0,0.0,180.58900,0.0,186.0,143.0,...,0.0,110.801000,25.0,0.0,0.0,42.637000,37.847000,0.00000,0.0,0.000000
1,2013-01-02,255.0,0.0,207.0,72092.0,0.0,26246.31900,0.0,74629.0,23381.0,...,0.0,20871.464028,17204.0,0.0,0.0,13975.884938,5338.111976,0.00000,0.0,1526.750002
2,2013-01-03,161.0,0.0,125.0,52105.0,0.0,18456.48002,0.0,55893.0,18001.0,...,0.0,16597.398113,12568.0,0.0,0.0,10674.393983,3591.388005,0.00000,0.0,1094.310994
3,2013-01-04,169.0,0.0,133.0,54167.0,0.0,16721.96901,0.0,52064.0,18148.0,...,0.0,21625.963055,11303.0,0.0,0.0,10772.515038,4472.965990,0.00000,0.0,1293.120995
4,2013-01-05,342.0,0.0,191.0,77818.0,0.0,22367.76108,0.0,70128.0,23082.0,...,0.0,20879.091050,16819.0,0.0,0.0,13475.009055,5830.073020,0.00000,0.0,1245.637004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1679,2017-08-11,441.0,7.0,351.0,189111.0,0.0,28204.48600,870.0,55682.0,44909.0,...,500.0,26944.092010,15978.0,587.0,654.0,25318.297990,5199.494021,118738.14300,3523.0,1272.615997
1680,2017-08-12,403.0,3.0,369.0,182318.0,0.0,27403.54800,849.0,57935.0,42854.0,...,483.0,17775.650112,15903.0,508.0,712.0,19134.510058,4573.465992,111788.35090,3644.0,1028.030006
1681,2017-08-13,481.0,9.0,433.0,202354.0,1.0,33262.75700,544.0,61757.0,50112.0,...,469.0,18633.015039,18188.0,541.0,741.0,20509.265004,4941.509018,125923.80240,3718.0,1118.047000
1682,2017-08-14,292.0,9.0,337.0,174832.0,0.0,28002.45700,594.0,55032.0,43050.0,...,457.0,16975.686040,15077.0,417.0,500.0,18597.508060,4647.375002,115257.59598,2826.0,970.679999


In [4]:
test_data=pd.read_csv('.gitignore/test.csv')
family_list=sorted(list(set(test_data['family'])))
n_prediction=len(test_data)/len(family_list)
n_prediction
target_test=list(set(test_data['date']))

In [5]:

from pandas import DataFrame
df = DataFrame()
df['price'] = timeseries['AUTOMOTIVE']
df['target'] = df['price'].shift(-1)
# df.dropna(inplace=True)
print(df)

      price  target
0       0.0   255.0
1     255.0   161.0
2     161.0   169.0
3     169.0   342.0
4     342.0   360.0
...     ...     ...
1679  441.0   403.0
1680  403.0   481.0
1681  481.0   292.0
1682  292.0   337.0
1683  337.0     NaN

[1684 rows x 2 columns]


In [5]:
# df2 = DataFrame()
# df2['price']=[1]
# df2['target']=[2]

In [6]:
def train_test_split(data,percentange):
    data=data.values
    n=int(len(data)*(1-percentange))
    return data[:n],data[n:]
train,test=train_test_split(df,0.001)

In [8]:
X=train[:,:-1]
y=train[:,-1]
X

array([[  0.],
       [255.],
       [161.],
       ...,
       [313.],
       [441.],
       [403.]])

In [108]:
model=XGBRegressor(objective='reg:squarederror',n_estimators=1000)
model.fit(X,y)

XGBRegressor(base_score=0.5, booster='gbtree', callbacks=None,
             colsample_bylevel=1, colsample_bynode=1, colsample_bytree=1,
             early_stopping_rounds=None, enable_categorical=False,
             eval_metric=None, gamma=0, gpu_id=-1, grow_policy='depthwise',
             importance_type=None, interaction_constraints='',
             learning_rate=0.300000012, max_bin=256, max_cat_to_onehot=4,
             max_delta_step=0, max_depth=6, max_leaves=0, min_child_weight=1,
             missing=nan, monotone_constraints='()', n_estimators=1000,
             n_jobs=0, num_parallel_tree=1, predictor='auto', random_state=0,
             reg_alpha=0, reg_lambda=1, ...)

In [109]:
test

array([[481., 292.],
       [292., 337.]])

In [110]:
val=np.array(test[0,0]).reshape(1,-1)
pred=model.predict(val)
print(pred[0])

400.33392


In [111]:
def xgb_predict(train,val):
    train=np.array(train)
    X,y=train[:,:-1],train[:,-1]
    model=XGBRegressor(objective='reg:squarederror',n_estimators=1000)
    model.fit(X,y)
    val=np.array(val).reshape(1,-1)
    pred=model.predict(val)
    return pred[0]

In [112]:
xgb_predict(train,test[0,0])

400.33392

In [113]:
# Walk-forward Validation
from sklearn.metrics import mean_squared_error

def validate(data,percentange):
    predictions=[]
    train,test=train_test_split(data,percentange)
    history=[x for x in train]
    
    for i in range(len(test)):
        test_X ,test_y= test[i,:-1],test[i,-1]
        pred=xgb_predict(history,test_X[0])
        predictions.append(pred)
        
        data.append(predictions)
        
        history.append(test[i])
        
    error=mean_squared_error(test[:,-1],predictions,squared=False)
    return error, test[:,-1], predictions,train

In [132]:
e,t,p,tr=validate(df,0.002)
t

array([403., 481., 292., 337.])

In [124]:
predictions=[]
train,test=train_test_split(df,0.001)
history=[x for x in train]

for i in range(17):
    error,test,predict,train=validate(df,0.001)
    df2['price'].append(test[1])
    df2['target'].append(predict[0])
    df.append(df2)

KeyError: 'price'

In [122]:
df

,price,target
0,0.0,255.0
1,255.0,161.0
2,161.0,169.0
3,169.0,342.0
4,342.0,360.0
...,...,...
1678,313.0,441.0
1679,441.0,403.0
1680,403.0,481.0
1681,481.0,292.0


In [24]:
predictions=[]
train,test=train_test_split(df,0.002)
history=[x for x in train]

for i in range(len(test)):
    test_X ,test_y= test[i,:-1],test[i,-1]
    pred=xgb_predict(history,test_X[0])
    predictions.append(pred)

    df.append(predictions)

    history.append(test[i])

In [25]:
pred

322.2859

In [26]:
test_data=pd.read_csv('.gitignore/test.csv')
test_data

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0
...,...,...,...,...,...
28507,3029395,2017-08-31,9,POULTRY,1
28508,3029396,2017-08-31,9,PREPARED FOODS,0
28509,3029397,2017-08-31,9,PRODUCE,1
28510,3029398,2017-08-31,9,SCHOOL AND OFFICE SUPPLIES,9


In [27]:
family_list=timeseries.columns
family_list=np.array(family_list)
family_list=family_list[1:]

In [28]:
# Walk-forward Validation
from sklearn.metrics import mean_squared_error

def validate(data,percentange):
    predictions=[]
    train,test=train_test_split(data,percentange)
    history=[x for x in train]
    
    for i in range((len(test_data)/len(family_list))+2):
        test_X ,test_y= test[i,:-1],test[i,-1]
        pred=xgb_predict(history,test_X[0])
        predictions.append(pred)
        
        history.append(test[i])
        
    error=mean_squared_error(test[:,-1],predictions,squared=False)
    return error, test[:,-1], predictions

In [ ]:
def create_model_fit(LSTM_unit=64, dropout=0.2) :
    model = Sequential()
    model.add(LSTM(units=LSTM_unit, input_shape=(window_size, 1)))
    model.add(Dense(32, activation='relu'))
    model.add(Dropout(dropout)) 
    model.add(Dense(1)) #output

    return model